# LangChain QuickStart Tutorial

I'm essentially following this QuickStart:

https://python.langchain.com/docs/get_started/quickstart

However, I've modified some of the statements and queries. I'm also using Azure OpenAI so some of the imports differ.

In [14]:
import dotenv
import os

# Use a local .env file to populate environment variables
dotenv.load_dotenv(override=True)

model_deployment = os.getenv('MODEL_DEPLOYMENT')
embedding_deployment = os.getenv('EMBEDDING_DEPLOYMENT')


This first cell creates a basic chain with a system prompt, the llm, and an output parser.
It then asks the LLM "Who is Martin Peck?" and prints the response.
It should give a response similar to the following:

> I'm sorry, but as a software engineer, I don't have specific information on individuals unless they are widely recognized in the field of software engineering or technology. Could you please provide more context?


In [15]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = AzureChatOpenAI(deployment_name=model_deployment)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a software engineer. When responding, be brief and to the point."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "Who is Martin Peck?"})


"I'm sorry, but as a software engineer, I don't have specific information on individuals unless they are significant in the field of technology or software development. I can't provide information about Martin Peck."

This next cell uses the WebBaseLoader (which, under the covers, uses BeautifulSoup) to scrape my blog "About" page

In [16]:
from langchain_community.document_loaders import WebBaseLoader

# scrape the web page
loader = WebBaseLoader("https://martinpeck.com/about/")
docs = loader.load()

Next, we prepare an embedding model (again, using the Azure OpenAI service), chunk the scraped web page, and then generate and store embeddings in a local FAISS vector store

In [17]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# prep the embedding model
embeddings = AzureOpenAIEmbeddings(azure_deployment=embedding_deployment)

# split the scraped web page into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# create embeddings and store them in a local FAISS vector store
vector = FAISS.from_documents(documents, embeddings)

Finally, we set up a chat template that allows us to inject both context and a question into the chat.
When we invoke this chain the LLM is given the context from the vector store, and this time when asked "Who is Martin Peck" should respond with something similar to the following:

> Martin Peck is a software engineer with over 20 years of experience in writing software and leading software development teams. He is currently a Software Engineering Manager at Microsoft, where he manages a team of Software Engineers. He has also worked for blinkbox and was previously the Director of Engineering for a not-for-profit called Code Club. In addition to his work in software, he also draws and publishes cartoons, some of which can be found for sale at Redbubble and Zazzle.

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Who is Martin Peck?"})
print(response["answer"])

Martin Peck is a software developer and team leader with over 20 years of professional experience. He is currently a Software Engineering Manager at Microsoft, leading a team within the Commercial Software Engineering (CSE) team. He has also worked for the company blinkbox, where he served as the Head of Client Device Development and Director of Engineering. Additionally, he was the Director of Engineering for a not-for-profit called Code Club, which aims to teach children about coding and technology. In his free time, he draws and publishes cartoons.


In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Is Martin a software engineer?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "for how long?"
})


{'chat_history': [HumanMessage(content='Is Martin a software engineer?'),
  AIMessage(content='Yes!')],
 'input': 'for how long?',
 'context': [Document(page_content='About Me | martinpeck.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nhome\n\n\nabout\n\n\ncartoons\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nmartinpeck.com\n\n         A blog, written by someone called Martin Peck\n    \n\n\n\n\n\nAbout Me\n\n\nHi. I’m Martin Peck. I’ve spent the last 20+ years getting paid for writing software and leading teams of software developers. When I’m not doing that I’m generally doing whatever my family tell me to do.\nYou can find in these other places:\n\nGitHub.com\nStack Overflow\nLinkedIn\nTwitter\n\nI also draw and publish cartoons. Some of my creations can be found on sale at Redbubble and Zazzle.\n\nMicrosoft (again)\nI’ve returned to Microsoft, and I’m currently a Software Engineering Manager at Microsoft, within the Commercial Software Engineering (CSE) team. I manage a team of